# Persistence

While with CPU/Mem virtualization and Concurrency, we've focussed alot fo our discussion on safely sharing resources, our major concern for file systems and disks will be robustness.

Our goal is to "permanently" store information so that it persists across reboots.

When we write a file, we want to ensure that what we wanted to write gets written.

We need information to persist across reboots, but also unexpected power offs.

An unexpected poweroff could corrupt the file system. We need to guarantee this can't happen.

## Intro to I/O Devices

Computers are organized as a hierarchy of devices

At the top is CPU/Mem bus.

Below this are other busses that connect to other devices.

Eg. Graphics cards, hard disks, keyboards/mice

Accessing any I/O device is atleast an order of magnitude slower than performing CPU computations.

Many I/O devices have their own microprocessors or even mini CPUs and they execute independently from the main CPU.

This allows the OS to delegate the work of performing the write. It makes requests to and from the hard disk.

These requests are issued by writing to a set of outward facing registers on the hard disk:
- COMMAND
- DATA
- STATUS

The OS likes to write to files in 4KB chunks. When it wants to write to a file, it will write the "write" command to the COMMAND register, then copy the 4KB of data to be written from memory onto the DATA register.

Then the hard disk will write that data to the actual disk.

STATUS is used by the hard disk to post its current status (busy, done).

**A Generic Protocol**

OS Writing to Hard Disk:

```
while(STATUS == busy):
    spin;
write data -> DATA
write command -> COMMAND
while(STATUS == busy):
    spin;
// check status, retrieve information, etc.
```

This would be horribly innefficient.

For efficiency, we use interrupts.

